### **Librerias necesarias**

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta, date
import numpy as np
import re
import os
import psycopg2
import json
from unidecode import unidecode

 ## **<u>Conectar a la API</u>** 

In [ ]:
# Leer el archivo de credenciales
with open('credentials.json') as file:
    credentials = json.load(file)

In [ ]:
#Parametros de la busqueda

query = '''ucrania OR Ukraine OR Ukraine OR Ucraina'''#palabras clave para efectuar la busqueda en API

today = datetime.today()
yesterday = today - timedelta(days=1) # Obtener la fecha del dia de ayer
from_date = yesterday.strftime('%Y-%m-%d')
results_limit = 100 
api_key = credentials['api_key']
languages = ['es', 'en', 'fr', 'it']

# Funcion para realizar la conexion a la API
def get_articles(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print('Error al conectarse a la API:', e)
        return []
    else:
        data = response.json()
        articles = data['articles']
        return articles

# Crear una lista de diccionarios con los datos de cada artículo.
articles_list = []
for lang in languages:
    url = f"https://newsapi.org/v2/everything?q={query}&language={lang}&from={from_date}&pageSize={results_limit}&apiKey={api_key}"
    articles = get_articles(url)
    for article in articles:
        article_data = {
            'author': article['author'],
            'title': article['title'],
            'description': article['description'],
            'url': article['url'],
            'publishedAt': article['publishedAt'],
            'source': article['source']['name'],
            'language': lang
                        }
        articles_list.append(article_data)


In [ ]:
#convertir la lista de diccionarios en un DataFrame
df = pd.DataFrame(articles_list)

### **Guardar en un .csv los datos de la API, sin procesar**

In [ ]:
#Se guardara un .csv distinto por dia.
current_folder = os.getcwd()
destination_folder = os.path.join(current_folder, 'Raw_csvs')
yesterday = date.today() - timedelta(days=1)
date_str = yesterday.strftime("%Y-%m-%d")
file_name = os.path.join(destination_folder, f"news {date_str}.csv")
df.to_csv(file_name, index=False)

In [ ]:
# Especifica la ruta del archivo CSV
ruta_archivo = 'C:\\Users\\juanm\\OneDrive\\Escritorio\\pythonProject\\Data. Eng --Coderhouse\\Raw_csvs\\news 2023-05-28.csv'

# Lee el archivo CSV en un DataFrame
df = pd.read_csv(ruta_archivo)

# Haz uso del DataFrame df para realizar las operaciones necesarias
# ...

# Imprime el DataFrame para verificar los datos cargados
df

## **<u>Limpiar los datos</u>**

#### ***Declarar algunas variables y funciones***

Creo 2 funciones para limpiar los textos, quitando simbolos, comas, tildes, parentesis, etc

In [ ]:
#Base para reordenar las columnas
column_order = ['source', 'title', 'description','author',  'publishedAt','language', 'url']

def eliminar_tildes(texto):
    caracteres_con_tilde = 'áéíóúÁÉÍÓÚ'
    caracteres_sin_tilde = 'aeiouAEIOU'
    reemplazar_map = str.maketrans(caracteres_con_tilde, caracteres_sin_tilde)
    texto_sin_tilde = unidecode(texto.translate(reemplazar_map))
    return texto_sin_tilde

def limpiar_celdas(dataframe, columnas):
    patron = r"[^a-zA-ZáéíóúÁÉÍÓÚüÜñÑ\d\s/]"
    for columna in columnas:
        dataframe[columna] = (
            dataframe[columna]
            .apply(lambda x: re.sub(patron, '', str(x)))
            .apply(eliminar_tildes)
        )
    return dataframe

columnas_limpiar = ['source', 'title', 'description', 'author']

#Esta funcion sirve para casos especificos. Borra lo que haya entre parentesis en una columna en particular
def eliminar_parentesis(columna):
    patron = r'\((.*?)\)'  # Patrón de expresión regular para encontrar el texto entre paréntesis
    columna = columna.apply(lambda x: re.sub(patron, '', x)).str.strip()
    # Eliminar el texto entre paréntesis y eliminar espacios en blanco
    return columna

In [ ]:
news_cleaned = df

In [ ]:
news_cleaned.columns

#### **Limpieza general del df**
Pasos a realizar:

- Eliminar duplicados (hay muchas notas que estan duplicadas pero difieren en el url. Tener eso en cuenta)
- Pasar palabras a lowercase, convertir las siglas de la columna lenguaje, reordenar las columnas
- Reemplazar valores vacios por Nan (despues se rellenan)

In [ ]:
news_cleaned = news_cleaned.drop_duplicates(subset=['source', 'title', 'description', 'author', 'publishedAt', 'language'])
news_cleaned = news_cleaned.apply(lambda x: x.astype(str).str.lower())
news_cleaned = news_cleaned.replace('', np.nan)
news_cleaned = news_cleaned.reindex(columns=column_order)
siglas_a_idiomas = {
    'en': 'english',
    'fr': 'french',
    'it': 'italian',
    'es': 'spanish'
}
news_cleaned['language'] = news_cleaned['language'].map(siglas_a_idiomas)


### **Limpieza especifica por columna**

En este orden: 
- Elimino parentesis y contenido de celdas en columna source.
- Reemplazo Nan values, y 'none' en columnas 'description' y 'author'. 
- En ambos casos, se reemplaza por lo que dice en la columna 'source'
- Muchos autores figuran asi: 'la nacion (Carlos pagni)'. Dejo solo los nombres que figuran entre parentesis.
- Ajusto manualmente unos pocos casos, usando replace
- Ejecuto la funcion limpiar_celdas en columnas seleccionadas
- Modifico la columna publishedAt, para que quede el formato fecha

In [ ]:
news_cleaned['source'] = eliminar_parentesis(news_cleaned['source'])
news_cleaned['description'] = news_cleaned['description'].fillna('none')
news_cleaned['author'] = news_cleaned['author'].fillna(news_cleaned['source'])
mask = news_cleaned['author'].eq('none')
news_cleaned['author'] = news_cleaned['author'].where(~mask, news_cleaned['source'])
mask = news_cleaned['author'].str.contains(r'\([^)]+\)')
news_cleaned.loc[mask, 'author'] = news_cleaned.loc[mask, 'author'].str.extract(r'\(([^)]+)\)', expand=False)
news_cleaned['author'] = (
    news_cleaned['author']
    .str.replace('rt en español\n', 'rt en español')
    .str.replace('https//www.facebook.com/bbcnews', 'bbcnews')
    .str.replace('rt en español , rt en español', 'rt en español'))
news_cleaned = limpiar_celdas(news_cleaned, columnas_limpiar)
news_cleaned['publishedAt'] = pd.to_datetime(news_cleaned['publishedAt'])
news_cleaned['publishedAt'] = news_cleaned['publishedAt'].dt.date

## **<u>Conectar con Amazon Redshift</u>**

In [ ]:
# Realizar conexion a Redshift mediante psycopg2
try:
    conn = psycopg2.connect(
    host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
    dbname=credentials['db_username'],
    user=credentials['db_password'],
    password='qjA21aB81Y',
    port='5439'
)
    print("Conexion con Redshift exitosa")
    
except Exception as e:
    print("Error al conectar con Redshift")
    print(e)


## **<u>Crear la tabla en Redshift</u>**

In [ ]:
# Crear un cursor para ejecutar sentencias SQL
cur = conn.cursor()

In [ ]:
# Eliminar la tabla si ya existe. Ejecutar solo en caso de error
#cur.execute('DROP TABLE IF EXISTS news_articles')

# Crear una tabla en Redshift, en caso de que no exista
table_name = 'news_articles'
cur.execute(f'''
CREATE TABLE IF NOT EXISTS {table_name} (
    id INT IDENTITY(1, 1),
    source VARCHAR(255),
    title VARCHAR(1000),
    description VARCHAR(1000),
    author VARCHAR(255),
    publishedAt DATE,
    language VARCHAR(7),
    url NVARCHAR(1000),
    CONSTRAINT unique_news UNIQUE (source, title, description, author)
)
''')


## **<u>Insertar los datos ya limpios en la tabla</u>**

Se procede a verificar si existen datos duplicados entre los que ya estan almacenados en la BD en Redshift, y los que estan por insertarse.
Tambien se crea un contador para tener seguimiento(en caso de quererlo) acerca de cuantas filas se dejan de lado, por ser duplicados

In [ ]:
duplicates_count = 0

# Verificar si cada fila ya existe antes de insertar los datos
for index, row in news_cleaned.iterrows():
    cur.execute(f'''
        SELECT COUNT(*) FROM {table_name}
        WHERE source = %s AND title = %s AND description = %s AND author = %s
    ''', (
        row['source'],
        row['title'],
        row['description'],
        row['author']
    ))
    count = cur.fetchone()[0]
    if count == 0:
        cur.execute(f'''
            INSERT INTO {table_name} (source, title, description, author, publishedAt, language, url)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        ''', (
            row['source'],
            row['title'],
            row['description'],
            row['author'],
            row['publishedAt'],
            row['language'],
            row['url']
        ))
    else:
        duplicates_count += 1

print(f"{duplicates_count} datos duplicados no fueron insertados.")

# Hacer commit de la transacción y cerrar la conexión
conn.commit()
conn.close()

In [ ]:
#En caso de que el codigo para insertar los datos en la tabla de redshift fallen, ejecuto:
#cur.execute('ROLLBACK')
